In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [45]:
dupl_columns = list(diabetes.columns)


mask = diabetes.duplicated(subset=dupl_columns)
diabet_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabet_duplicates.shape[0]}')

Число найденных дубликатов: 10


In [46]:
diabet_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabet_dedupped.shape[0]}')

Результирующее число записей: 768


In [47]:
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [48]:
display(diabetes.isnull().tail())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
773,False,False,False,False,False,False,False,False,False,False
774,False,False,False,False,False,False,False,False,False,False
775,False,False,False,False,False,False,False,False,False,False
776,False,False,False,False,False,False,False,False,False,False
777,False,False,False,False,False,False,False,False,False,False


In [49]:
diabetes.loc[diabetes['Glucose'] == 0, 'Glucose'] = None
diabetes.loc[diabetes['SkinThickness'] == 0, 'SkinThickness'] = None
diabetes.loc[diabetes['Insulin'] == 0, 'Insulin'] = None
diabetes.loc[diabetes['BloodPressure'] == 0, 'BloodPressure'] = None
diabetes.loc[diabetes['BMI'] == 0, 'BMI'] = None
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98.0,58.0,33.0,190.0,34.0,0.430,43,0,Female
1,2,112.0,75.0,32.0,NaN,35.7,0.148,21,0,Female
2,2,108.0,64.0,NaN,NaN,30.8,0.158,21,0,Female
3,8,107.0,80.0,NaN,NaN,24.6,0.856,34,0,Female
4,7,136.0,90.0,NaN,NaN,29.9,0.210,50,0,Female


In [50]:
diabetes.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                36
SkinThickness               232
Insulin                     380
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
Gender                        0
dtype: int64

In [51]:
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.30:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Outcome: 65.55% одинаковых значений
Gender: 100.0% одинаковых значений


In [52]:
information_sber_data = diabetes.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_sber_data.shape[1]}')

Результирующее число признаков: 8


In [53]:
m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
print(diabetes.shape[0])

742


In [55]:
#создаем копию исходной таблицы
fill_data = diabetes.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'Glucose': diabetes['Glucose'].median(),
    'BMI': diabetes['BMI'].median()
}
#заполняем пропуски в соответствии с заявленным словарем
diabetes = diabetes.fillna(values)
#выводим результирующую долю пропусков
diabetes.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.000000
BloodPressure               0.000000
SkinThickness               0.000000
Insulin                     0.463612
BMI                         0.000000
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
Gender                      0.000000
dtype: float64

In [56]:
null_data = diabetes.isnull().sum()
cols = null_data[null_data>0].index
for col in cols:
    diabetes[col] = diabetes[col].fillna(diabetes[col].median())
print(diabetes['SkinThickness'].mean().round(1))

29.1


In [57]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])

53


In [58]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])

4


In [59]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_log.shape[0])

29
